<a href="https://colab.research.google.com/github/shaunck96/LangGraph-GPT4-TAVELI/blob/main/LangGraph_%2B_GPT4_%2B_TAVELI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langgraph langchain_openai langchainhub tavily-python

  Using cached tavily_python-0.3.0-py3-none-any.whl (5.4 kB)
INFO: pip is looking at multiple versions of tavily-python to determine which version is compatible with other requirements. This could take a while.
  Using cached tavily_python-0.2.9-py3-none-any.whl (5.4 kB)
  Using cached tavily_python-0.2.8-py3-none-any.whl (5.3 kB)
  Using cached tavily_python-0.2.7-py3-none-any.whl (5.2 kB)
  Using cached tavily_python-0.2.6-py3-none-any.whl (4.8 kB)
  Using cached tavily_python-0.2.5-py3-none-any.whl (4.8 kB)
  Using cached tavily_python-0.2.4-py3-none-any.whl (4.7 kB)
  Using cached tavily_python-0.2.3-py3-none-any.whl (4.7 kB)
INFO: pip is looking at multiple versions of tavily-python to determine which version is compatible with other requirements. This could take a while.
  Using cached tavily_python-0.2.2-py3-none-any.whl (4.6 kB)


In [2]:
pip install -U langchain-community

In [21]:
from google.colab import userdata
import os
from langchain import hub #pull prompt templates from hub, tracing
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [22]:
#Agent Definition

### Generating openai_function_Agent

In [23]:
tools = [TavilySearchResults(max_results=1)]
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-4-1106-preview")

In [24]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(

In [25]:
agent_runnable = create_openai_functions_agent(llm, tools, prompt)
agent_runnable

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history',

In [26]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.agents import AgentFinish

agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

In [27]:
agent

RunnableAssign(mapper={
  agent_outcome: RunnableAssign(mapper={
                   agent_scratchpad: RunnableLambda(lambda x: format_to_openai_function_messages(x['intermediate_steps']))
                 })
                 | ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],

In [28]:
##Service Function

def execute_tools(data):
  agent_action = data.pop('agent_outcome')
  tools_to_use = {t.name: t for t in tools}[agent_action.tool]
  observation = tools_to_use.invoke(agent_action.tool_input)
  data['intermediate_steps'].append((agent_action, observation))
  return data

In [29]:
def should_continue(data):
  if isinstance(data['agent_outcome'], AgentFinish):
    return "exit"
  else:
    return "continue"

In [30]:
from langgraph.graph import END, Graph

In [31]:
workflow = Graph()
workflow.add_node("agent",agent)
workflow.add_node("tools",execute_tools)

In [32]:
workflow.set_entry_point("agent")

In [33]:
workflow.add_conditional_edges(
    "agent", #start node
    should_continue,
    {
        "continue":"tools",
        "exit":END
    }
)

In [34]:
workflow.add_edge("tools","agent")

In [35]:
chain = workflow.compile()

In [37]:
##Execution
chain.invoke({"input":"How to become an investment advisor in 5 months", "intermediate_steps":[]})

{'input': 'How to become an investment advisor in 5 months',
 'intermediate_steps': [],
 'agent_outcome': AgentFinish(return_values={'output': "Becoming an investment advisor in a relatively short period, such as 5 months, will require a structured approach, dedication, and strategic planning. Here are the general steps you could follow to become an investment advisor within this timeframe:\n\n1. **Understand the Role and Requirements:**\n   - Research what an investment advisor does, the services they offer, and the regulatory requirements in your country or region (such as licensing, education, and experience).\n\n2. **Educational Foundation:**\n   - Ensure you have the necessary educational background. This typically includes a bachelor's degree in finance, economics, business, or a related field. If you already have a degree, you can focus on acquiring specific knowledge related to investment advising.\n   - If you do not have a degree, you might consider accelerated courses, onlin